# Function Call Decorators

Another way to add features to your function calling would be via decorators,

LionAGI allows you to use the following for any sync and async functions

- `cache`   : keep the result in memory, if encountered same parameter input, will give output directly
- `throttle`    : set the time between each consecutive function call
- `map`
- `compose`     : apply multiple functions: The output of each function is passed as the input to the next, in the order they are provided
- `pre_post_process`    
- `filter`
- `reduce`  : reduces the results of a function to a single value using the specified reduction function

the following decorators only works for async functions
- `max_concurrency`
- `timeout` 
- `retry`
- `default`

In [1]:
import asyncio

import lionagi as li
from lionagi import CallDecorator as cd

the decorator works with both sync and async functions

### Sync and Async

`@cd.cache`

keep function result in memory

In [2]:
# sync
import time

@cd.cache
def square_data(x):
    time.sleep(0.5)
    return x**2


result, elapse = await li.tcall(x=10_000, func=square_data, delay=0, timing=True)
print(f"result: {result:_}, elapse: {elapse:.03f} seconds")

result, elapse = await li.tcall(x=10_000, func=square_data, delay=0, timing=True)
print(f"result: {result:_}, elapse: {elapse:.03f} seconds")

result: 100_000_000, elapse: 0.504 seconds
result: 100_000_000, elapse: 0.000 seconds


In [3]:
# async
@cd.cache
async def asquare_data(x):
    await asyncio.sleep(0.5)
    return x * x

result, elapse = await li.tcall(x=10_000, func=asquare_data, delay=0, timing=True)
print(f"result: {result:_}, elapse: {elapse:.03f} seconds")

result, elapse = await li.tcall(x=10_000, func=asquare_data, delay=0, timing=True)
print(f"result: {result:_}, elapse: {elapse:.03f} seconds")

result: 100_000_000, elapse: 0.501 seconds
result: 100_000_000, elapse: 0.000 seconds


`@cd.filter`

In [4]:
# filters can be used to filter the output upon the predicate
@cd.filter(predicate=lambda y: y<10)
def square_data(x):
    return [0, x**2]
    
print(square_data(5))

# async
@cd.filter(predicate=lambda y: y>10)
async def asquare_data(x):
    return [0, x**2]

print(await asquare_data(5))

[0]
[25]


`@cd.throttle` 

In [5]:
# sync
# this block should take 0.8 seconds to run, because for 5 elements, there are 4 intervals of 0.2 seconds
@cd.throttle(0.2)
def square_data(x):
    return(x**2)

print(li.lcall(range(5), square_data))

[0, 1, 4, 9, 16]


In [6]:
# async
@cd.throttle(0.2)
async def asquare_data(x):
    return(x**2)

print(await li.alcall(range(5), asquare_data))

[0, 1, 4, 9, 16]


`@cd.pre_post_process`

In [7]:
# this decorator will apply the preprocess and postprocess function to the 
# input and output respectively
# sync
@cd.pre_post_process(preprocess=lambda x: x+1, postprocess=lambda x: x**2)
def get_5(x):
    return x

print(get_5(5))

# async
@cd.pre_post_process(preprocess=lambda x: x-1, postprocess=lambda x: x**3)
async def get_5(x):
    return x

print(await get_5(5))

36
64


`@cd.compose`

In [8]:
f1 = lambda x: x+1
f2 = lambda x: x*2

# this decorator will compose the functions into a single funtion
@cd.compose(f1, f2)
def increment_and_double(x):
    return x

result = increment_and_double(3)        # first increment_and_double return 3 to f1
print(result)                           # f1 returns (3+1), then that becomes f2 input, then f2 outputs 2*4 = 8

8


In [9]:
async def f1(coroutine_):        # here the coroutine is the function that is being decorated on, 
    a = await coroutine_
    return a+1

async def f2(x):        # the output from f1 is an integer
    return x*2

@cd.compose(f1, f2)                         # meaning f2( f1( func(x)))where func(x) is a coroutine
async def aincrement_and_double(x):
    return x

result = await aincrement_and_double(3)     # here because this is an async function, we need to await it in f1
print(result)

8


`@cd.map`

In [10]:
# this decorator allows you to map a function to a list of inputs
@cd.map(lambda x: x * x)
def square_numbers(numbers):
    return numbers

numbers = [1, 2, 3, 4, 5]
squared_numbers = square_numbers(numbers)
print(squared_numbers)

[1, 4, 9, 16, 25]


In [11]:
# this decorator allows you to map a function to a list of inputs
@cd.map(lambda x: x * x + 1)
async def square_numbers(numbers):
    return numbers

numbers = [1, 2, 3, 4, 5]
await square_numbers(numbers)

[2, 5, 10, 17, 26]

`@cd.reduce`

In [12]:
# this decorator reduces the sequence to a single value
@cd.reduce(lambda x, y: x + y, 0)
def sum_numbers(numbers):
    return numbers

numbers = [1, 2, 3, 4, 5]
sum_numbers(numbers)

15


In [13]:
@cd.reduce(lambda x, y: x - y, 0)
async def minus_numbers(numbers):
    return numbers

numbers = [1, 2, 3, 4, 5]
await minus_numbers(numbers)

-15

### Async only

`cd.retry`

In [14]:
# this decorator will retry the function call if it fails
# initial_delay is the time in seconds to wait before the first retry
# should take 0.7 second to run - first delay 0.1, second delay 0.2, third delay 0.4

@cd.retry(retries=3, delay=0.1, backoff_factor=2)
async def retry_function(attempt):
    if attempt < 3:
        raise ValueError("Simulated error")
    return "Success"

try:
    result = await retry_function(1)
    print(result)
except ValueError as e:
    print(f"Function call failed after retries: {e}")

An error occurred: Simulated error
An error occurred: Simulated error
An error occurred: Simulated error
Function call failed after retries: Simulated error


`@cd.timeout`

In [15]:
# this decorator sets a timeout limit in seconds for the function to run
@cd.timeout(0.5)
async def timeout_function():
    await asyncio.sleep(2)
    return "Completed"

try:
    result = await timeout_function()
    print(result)
except Exception as e:
    print(f"Function call timed out: {e}")

An error occurred: 
Function call timed out: 


`@cd.max_concurrency`


In [16]:
# this should take 1.5 seconds to run
# limits the amount of concurrent function calls, in this case at most 2 calls at a time
@cd.max_concurrency(limit=2)
async def limited_concurrency_function(x):
    await asyncio.sleep(0.5)
    return x * 2

async def run_max_concurrency_example():
    results = await li.alcall(range(5), limited_concurrency_function)
    print(results)

await run_max_concurrency_example()

[0, 2, 4, 6, 8]


`cd.default`

In [17]:
# this decorator will return a default value if the function fails
@cd.default(default_value="Default Result")
async def default_function(may_fail):
    if may_fail:
        raise ValueError("Simulated error")
    return "Success"

result = await default_function(True)
print(result)

An error occurred: Simulated error
Default Result
